<a href="https://colab.research.google.com/github/Sarthak9504/SY-Sem1-CourseProjects/blob/main/Marathi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras import layers
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import tensorflow as tf
import re
from google.colab import drive
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_excel("/content/tweets-train-cleaned.xlsx", index_col='Unnamed: 0')
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

df.loc[~df['tweet'].isnull()]

,tweet,label
0,ज्येष्ठ पत्रकार अनंत दीक्षित यां निधन वृत्त दु...,-1
1,सर्वोच् न्ल निर्देश डावलून पुण पोलिस प्र आनंद ...,-1
2,उद्ध ठाकर भाजपासोबत युत शिसैनिक अमान िश्ासघ फस...,-1
3,आपल समाज खूप मोठ त्य घटक अंतर्भूत त या दिव्यां...,1
4,बलात्कार बदल बलात्कार घेतल पाहिज म्हणणार सावरक...,-1
...,...,...
12109,त्याचबरोबर काज व्यावसायिकां मागील कालावध व्हॅट...,1
12110,सरकार कर्जमाफ योज लाग करुन सुमार वर्ष काळ उलटू...,-1
12111,लोकश मतदार खऱ्य अर्थ सत्ताधार असत धमकािण अथ प्...,-1
12112,छत्रपत शिवाज महाराज ंच् नाव शिवसेने स्थाप ंन श...,-1


In [ ]:
df.shape

(12114, 2)

In [ ]:
df.loc[~df['tweet'].isnull()]

,tweet,label
0,ज्येष्ठ पत्रकार अनंत दीक्षित यां निधन वृत्त दु...,-1
1,सर्वोच् न्ल निर्देश डावलून पुण पोलिस प्र आनंद ...,-1
2,उद्ध ठाकर भाजपासोबत युत शिसैनिक अमान िश्ासघ फस...,-1
3,आपल समाज खूप मोठ त्य घटक अंतर्भूत त या दिव्यां...,1
4,बलात्कार बदल बलात्कार घेतल पाहिज म्हणणार सावरक...,-1
...,...,...
12109,त्याचबरोबर काज व्यावसायिकां मागील कालावध व्हॅट...,1
12110,सरकार कर्जमाफ योज लाग करुन सुमार वर्ष काळ उलटू...,-1
12111,लोकश मतदार खऱ्य अर्थ सत्ताधार असत धमकािण अथ प्...,-1
12112,छत्रपत शिवाज महाराज ंच् नाव शिवसेने स्थाप ंन श...,-1


In [ ]:
df_v=pd.read_excel("/content/tweets-valid-cleaned.xlsx", index_col='Unnamed: 0')

In [ ]:
df_v.loc[~df_v['tweet'].isnull()]

,tweet,label
0,कोर काळ पंतप्रधान मा नरेंद्र मोदीज य समाज घटक ...,1
1,पुण राष्ट्रसे दल अभ्सिक िभाग ंच् संयुक्त िद्यम...,1
2,पुण क्रांत ज्योत कार्यक्रम जेष्ठ समाजवाद ने मा...,0
3,आद्यक्रांतिकारक उमाजीराज नाईक य जयंतीनिमित्त व...,1
4,महाराष्ट्र मुख्यमंत्र शपथ घेतल्याबद्दल शिवस पक...,1
...,...,...
1495,पोलिसभरत प्रक्रिय जु निकषांनुसार व्व याबाबत मा...,1
1496,निध उद्ध्वस्त कुटुंबांमधील मुल शिक्षण वापरल जा...,1
1497,िकासा फक्त घोषणाबाज जाहिरातबाज राज्य जनते दिशा...,0
1498,विधानसभेत राष्ट्रीय जनगण ओबीसीं इतर मागासवर्गी...,0


In [ ]:
df_v.shape

(1500, 2)

In [ ]:
df_t = pd.read_excel("/content/tweets-test-cleaned.xlsx", index_col='Unnamed: 0')

In [ ]:
df_t.shape

In [ ]:
max(df_t['tweet'].str.len())

240

In [ ]:
max(df_v['tweet'].str.len())

In [ ]:
vocab_df=pd.concat([df_v['tweet'], df.loc[~df['tweet'].isnull(), 'tweet']], axis = 0)

In [ ]:
vocab_df.isnull().sum()

0

In [ ]:
set2=set()
for row in vocab_df.iteritems():
  for words in row[1].split():
    set2.add(words)
len(set2)

<ipython-input-17-c0937d06b7d9>:2: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for row in vocab_df.iteritems():


38045

In [ ]:

vocab_size=40000


In [ ]:
from collections import Counter
def counter_word(text_col):
  count=Counter()
  for text in text_col.values:
    for word in text.split():
      count[word]+=1
  return count

In [ ]:

counter = counter_word(vocab_df)

len(counter)


38045

In [ ]:

from tensorflow.keras.preprocessing.text import Tokenizer
tokinizer=Tokenizer(num_words=40000)
tokinizer.fit_on_texts(df.loc[~df['tweet'].isnull(), 'tweet'])

In [ ]:

word_index = tokinizer.word_index

In [ ]:
word_index

In [ ]:
train_sequences = tokinizer.texts_to_sequences(df.loc[~df['tweet'].isnull(), 'tweet'])

In [ ]:
test_sequences = tokinizer.texts_to_sequences(df_t['tweet'].values)


In [ ]:
val_sequences = tokinizer.texts_to_sequences(df_v['tweet'].values)


In [ ]:

print(df['tweet'][0])
print(train_sequences[0])

ज्येष्ठ पत्रकार अनंत दीक्षित यां निधन वृत्त दुखद दशकं परखड लेखण पत्रकारितेत अमूल्य योगदान दि दीक्षित यां मार्गदर्शनाखाल पत्रकारां पिढ घडल अनंत दीक्षित य भावपूर्ण श्रद्धांजल त्य परिवार दुख सहभाग आहोत
[83, 132, 1800, 4171, 4, 77, 795, 1676, 8272, 2692, 2387, 10711, 2693, 268, 19, 4171, 4, 1864, 5150, 994, 1732, 1800, 4171, 1, 62, 39, 14, 1733, 1230, 97, 236]


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train_padded=pad_sequences(train_sequences, maxlen=250, padding='post', truncating='post')
val_padded=pad_sequences(val_sequences, maxlen=250, padding='post', truncating='post')
test_padded=pad_sequences(test_sequences, maxlen=250, padding='post', truncating='post')

In [ ]:
print(train_padded.shape, val_padded.shape, test_padded.shape)

(12112, 250) (1500, 250) (2250, 250)


In [ ]:
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report
from keras.optimizers import Adam
model = Sequential()

In [ ]:
model.add(layers.Embedding(40000, 32, input_length=250))
model.add(layers.Dense(256, activation='gelu'))
model.add(layers.Flatten())
model.add(layers.Dense(3, activation='softmax'))

In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])


In [ ]:
train_label_categorical = to_categorical(df.loc[~df['tweet'].isnull(), 'label'], num_classes=3)


val_label_categorical = to_categorical(df_v['label'].values, num_classes=3)


test_label_categorical = to_categorical(df_t['label'].values, num_classes=3)

In [ ]:

val_label_categorical

array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [ ]:
train_label_categorical

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [ ]:
tf.config.run_functions_eagerly(True)
model.fit(train_padded, to_categorical(df.loc[~df['tweet'].isnull(), 'label'].values, num_classes=3), epochs=5, validation_data=(val_padded, to_categorical(df_v['label'].values, num_classes=3)), verbose=2)

Epoch 1/5
379/379 - 13s - loss: 0.0042 - accuracy: 0.9987 - val_loss: 1.2877 - val_accuracy: 0.7547 - 13s/epoch - 35ms/step
Epoch 2/5
379/379 - 14s - loss: 0.0038 - accuracy: 0.9986 - val_loss: 1.3346 - val_accuracy: 0.7540 - 14s/epoch - 36ms/step
Epoch 3/5
379/379 - 14s - loss: 0.0031 - accuracy: 0.9987 - val_loss: 1.3794 - val_accuracy: 0.7520 - 14s/epoch - 37ms/step
Epoch 4/5
379/379 - 19s - loss: 0.0032 - accuracy: 0.9985 - val_loss: 1.4056 - val_accuracy: 0.7527 - 19s/epoch - 50ms/step
Epoch 5/5
379/379 - 13s - loss: 0.0028 - accuracy: 0.9989 - val_loss: 1.4499 - val_accuracy: 0.7493 - 13s/epoch - 35ms/step


In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.6 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=3d34f7044ca93b6a40212667ace8f0db5437adb86ce5bbb193ee754613d44028
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from transformers import AutoTokenizer, BertTokenizer, TFBertModel, AutoModel, TFAutoModel, TFAutoModelForSequenceClassification

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/indic-sentence-bert-nli")

In [ ]:
df = pd.read_excel("/content/tweets-train-cleaned.xlsx", index_col='Unnamed: 0')
df_v = pd.read_excel("/content/tweets-valid-cleaned.xlsx", index_col='Unnamed: 0')
df_t = pd.read_excel("/content/tweets-test-cleaned.xlsx", index_col='Unnamed: 0')


In [ ]:
train_inputs = tokenizer(df.loc[~df['tweet'].isnull(), 'tweet'].tolist(), padding=True, truncation=True, max_length=250, return_tensors="tf")
val_inputs = tokenizer(df_v['tweet'].tolist(), padding=True, truncation=True, max_length=250, return_tensors="tf")
test_inputs = tokenizer(df_t['tweet'].tolist(), padding=True, truncation=True, max_length=250, return_tensors="tf")

In [ ]:
train_labels = df.loc[~df['tweet'].isnull(), 'label'].values
val_labels = df_v['label'].values
test_labels = df_t['label'].values

In [ ]:
num_classes = 3  # Number of sentiment classes
train_label_categorical = to_categorical(train_labels, num_classes=num_classes)
val_label_categorical = to_categorical(val_labels, num_classes=num_classes)
test_label_categorical = to_categorical(test_labels, num_classes=num_classes)

In [ ]:
model = SentenceTransformer("l3cube-pune/indic-sentence-bert-nli")

In [ ]:
train_embeddings = model.encode(df.loc[~df['tweet'].isnull(), 'tweet'].tolist(), show_progress_bar=True)
val_embeddings = model.encode(df_v['tweet'].tolist(), show_progress_bar=True)
test_embeddings = model.encode(df_t['tweet'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/379 [00:00<?, ?it/s]

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

Batches:   0%|          | 0/71 [00:00<?, ?it/s]

In [ ]:
classification_model = tf.keras.Sequential([
    Dense(256, activation='relu', input_shape=(train_embeddings.shape[1],)),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate=0.0001)
classification_model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])


In [ ]:
classification_model.fit(
    train_embeddings,
    train_label_categorical,
    epochs=5,
    batch_size=32,
    validation_data=(val_embeddings, val_label_categorical),
    verbose=2
)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/5
379/379 - 17s - loss: 0.9665 - accuracy: 0.6849 - val_loss: 0.8165 - val_accuracy: 0.7240 - 17s/epoch - 46ms/step
Epoch 2/5
379/379 - 18s - loss: 0.7492 - accuracy: 0.7200 - val_loss: 0.6713 - val_accuracy: 0.7453 - 18s/epoch - 46ms/step
Epoch 3/5
379/379 - 10s - loss: 0.6654 - accuracy: 0.7322 - val_loss: 0.6194 - val_accuracy: 0.7547 - 10s/epoch - 26ms/step
Epoch 4/5
379/379 - 11s - loss: 0.6351 - accuracy: 0.7383 - val_loss: 0.5988 - val_accuracy: 0.7593 - 11s/epoch - 29ms/step
Epoch 5/5
379/379 - 11s - loss: 0.6189 - accuracy: 0.7436 - val_loss: 0.5864 - val_accuracy: 0.7613 - 11s/epoch - 29ms/step


In [ ]:
test_loss, test_acc = classification_model.evaluate(test_embeddings, test_label_categorical)
print("Test accuracy:", test_acc)

71/71 [==============================] - 2s 33ms/step - loss: 0.6033 - accuracy: 0.7551
Test accuracy: 0.7551110982894897


In [ ]:
predictions = classification_model.predict(test_embeddings)
predicted_labels = np.argmax(predictions, axis=1)

71/71 [==============================] - 0s 5ms/step


In [ ]:
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       750
           0       0.71      0.72      0.72       750
           1       0.79      0.71      0.75       750
           2       0.00      0.00      0.00         0

    accuracy                           0.48      2250
   macro avg       0.38      0.36      0.37      2250
weighted avg       0.50      0.48      0.49      2250



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

y_train = to_categorical(df.loc[~df['tweet'].isnull(), 'label'], num_classes=3)
y_val = to_categorical(df_v['label'], num_classes=3)